# 00. データ前処理（Preprocessing / ETL）

## 目的
- raw（元データ）を、分析しやすい表形式に整形して processed に保存する
- Excelの「印刷用レイアウト（タイトル行・空行・結合セル）」をデータ形式に変換する

## 出力
- data/processed/ に加工済みCSVを出力する
- 
■ 品質確認ルール
・47都道府県存在すること
・prefecture に集計行が含まれないこと
・主要列に欠損がないこと

In [1]:
import pandas as pd
from pathlib import Path

In [ ]:
# ==========================================
# 1. 設定・パス定義
# ==========================================
BASE_DIR = Path("..")  # プロジェクトルート（ワークフローの基準ディレクトリ）を定義
RAW_DIR = BASE_DIR/ "data" / "raw"  # データレイクのRaw層（ソースデータの不変保管領域）を指定

# 保存先ディレクトリ確保
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)  # 再現性確保のため、依存ディレクトリを自動生成（冪等性担保）

In [ ]:
# ==========================================
# 2. ベースデータ（PDF由来）の読み込み
# ==========================================
print("Loading base data...")  # ETLパイプライン実行ログ（処理ステータスの可観測性を確保）

# ファイルパス
path_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"
path_night = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"  # 夜勤負荷データのRawソース位置を定義

# 読み込みと結合
try:  # ファイル欠損時にパイプラインが静かに失敗しないように例外処理を実装
    df_turnover = pd.read_csv(path_turnover)  # Raw層から離職率データをDataFrameとしてロード
    df_night = pd.read_csv(path_night)  # Raw層から夜勤負荷データをDataFrameとしてロード

    # 都道府県(prefecture)で結合して master の初期状態を作成
    df_master = pd.merge(df_turnover, df_night, on="prefecture", how="outer")  # 主キー(prefecture)でスキーマ統合（欠損保持のため外部結合を採用）

    # 簡易クレンジング（「計」や「全国」などの集計行を除外）
    exclude_targets = ["計", "全国", "未回答", "無回答・不明"]  # 分析単位を歪める非正規レコードの除外対象を定義
    df_master = df_master[~df_master["prefecture"].isin(exclude_targets)].copy()  # 非分析対象レコードをフィルタリングし、副作用防止のためコピー生成
    
    




In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# ===============================
# Path Settings
# ===============================

BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

PROCESSED_DIR.mkdir(exist_ok=True)


In [3]:
# ===============================
# Load Raw Data
# ===============================

df_turnover = pd.read_excel(RAW_DIR / "日本看護協会_病院看護実態調査_2025.xlsx")

df_rent = pd.read_excel(RAW_DIR / "総務省_住宅土地統計調査_家賃_2024.xlsx")

df_home = pd.read_excel(RAW_DIR / "総務省_住宅土地統計調査_持ち家率_2024.xlsx")

df_job = pd.read_excel(RAW_DIR / "厚労省_賃金構造基本統計調査_有効求人倍率_2024.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/日本看護協会_病院看護実態調査_2025.xlsx'

In [ ]:
list((Path("..") / "data" / "raw").glob("*"))



In [ ]:
from pathlib import Path

BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"

pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"
pdf_path.exists(), pdf_path

In [ ]:
import pdfplumber

with pdfplumber.open(pdf_path) as pdf:
    first_text = (pdf.pages[0].extract_text() or "")[:500]
first_text


In [ ]:
import pdfplumber

targets = ["表 8", "表8", "表 50", "表50"]
hits = {t: [] for t in targets}

with pdfplumber.open(pdf_path) as pdf:
    for i, p in enumerate(pdf.pages):
        text = p.extract_text() or ""
        for t in targets:
            if t in text:
                hits[t].append(i)

hits

In [ ]:
import pdfplumber

with pdfplumber.open(pdf_path) as pdf:
    print(pdf.pages[39].extract_text()[:1000])


In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    print(pdf.pages[108].extract_text()[:1000])


In [ ]:
import pdfplumber
import pandas as pd
from pathlib import Path

BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"
pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"

P_TURNOVER = 73  # PDF 74ページ
P_NIGHT    = 108 # PDF 109ページ

def preview_tables(page_idx: int, n_preview: int = 2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_idx]
        tables = page.extract_tables()
    print(f"page_idx={page_idx} tables={len(tables)}")
    for i, t in enumerate(tables[:n_preview]):
        df = pd.DataFrame(t)
        print(f"\n--- table {i} shape={df.shape} ---")
        print(df.head(8))
    return tables

tables_turnover = preview_tables(P_TURNOVER)
tables_night = preview_tables(P_NIGHT)


In [ ]:
def pick_table_by_keyword(tables, keyword="都道府県"):
    for t in tables:
        s = "\n".join(["\t".join([str(x) for x in row if x is not None]) for row in t])
        if keyword in s:
            return t
    return None

t_turnover = pick_table_by_keyword(tables_turnover, "都道府県")
t_night = pick_table_by_keyword(tables_night, "都道府県")

t_turnover is not None, t_night is not None


In [ ]:
import pdfplumber
import pandas as pd
from pathlib import Path

BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"
pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"

P_TURNOVER = 73   # PDF 74ページ（表8）
P_NIGHT    = 108  # PDF 109ページ（表50）

def show_tables(page_idx: int, max_tables: int = 10, head_rows: int = 12):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_idx]
        tables = page.extract_tables()
    print(f"\n=== page_idx={page_idx} tables_found={len(tables)} ===")
    for i, t in enumerate(tables[:max_tables]):
        df = pd.DataFrame(t).replace({None:""}).applymap(lambda x: str(x).strip())
        print(f"\n--- table {i} shape={df.shape} ---")
        print(df.head(head_rows).to_string(index=False))
    return tables

tables_turnover = show_tables(P_TURNOVER)
tables_night = show_tables(P_NIGHT)

In [ ]:
import pdfplumber
import pandas as pd
import re
from pathlib import Path

BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"

pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"

P_TURNOVER = 73   # PDF 74ページ（表8）
P_NIGHT    = 108  # PDF 109ページ（表50）

def extract_first_table(page_idx: int) -> pd.DataFrame:
    with pdfplumber.open(pdf_path) as pdf:
        t = pdf.pages[page_idx].extract_tables()[0]
    df = pd.DataFrame(t).replace({None: ""})
    df = df.map(lambda x: str(x).strip())  # applymap非推奨対応
    return df

def pct_to_float(x: str):
    """'11.3%' -> 11.3 / '' -> NaN"""
    if x is None:
        return pd.NA
    s = str(x).strip().replace("％", "%")
    if s == "":
        return pd.NA
    s = s.replace("%", "")
    try:
        return float(s)
    except:
        return pd.NA

# ----------------------------
# 1) Turnover (表8)
# ----------------------------
df_t = extract_first_table(P_TURNOVER)

# 先頭2行はヘッダ行、3行目以降がデータ（計/都道府県）
data_t = df_t.iloc[2:].copy()

# 列位置はあなたの出力で確定済み：
# 0: 都道府県（計/北海道...）
# 2: 正規雇用 離職率
# 4: 新卒 離職率
# 6: 既卒 離職率
turnover = pd.DataFrame({
    "prefecture": data_t[0],
    "turnover_total": data_t[2],
    "turnover_new_grad": data_t[4],
    "turnover_experienced": data_t[6],
})

# 「計」を除外
turnover = turnover[turnover["prefecture"] != "計"].copy()

# %を数値へ
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    turnover[c] = turnover[c].map(pct_to_float)

# ----------------------------
# 2) Night shift (表50)
# ----------------------------
df_n = extract_first_table(P_NIGHT)

# 1行目がヘッダ、2行目以降がデータ（計/都道府県）
data_n = df_n.iloc[1:].copy()

# 列位置（あなたの出力で確定済み）
# 0: 都道府県（計/北海道...）
# 5: 72時間を超える夜勤者率
night = pd.DataFrame({
    "prefecture": data_n[0],
    "night_shift_72h_plus": data_n[5],
})

night = night[night["prefecture"] != "計"].copy()
night["night_shift_72h_plus"] = night["night_shift_72h_plus"].map(pct_to_float)

# ----------------------------
# 3) Save as CSV (rawに保存)
# ----------------------------
out_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"
out_night = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"

turnover.to_csv(out_turnover, index=False, encoding="utf-8-sig")
night.to_csv(out_night, index=False, encoding="utf-8-sig")

turnover.head(), night.head(), out_turnover, out_night


In [ ]:
print(len(turnover), turnover.isna().sum())
print(len(night), night.isna().sum())

In [ ]:
df_turnover = pd.read_csv(RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv")
df_night = pd.read_csv(RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv")

df_turnover.head()
df_night.head()
print("=== turnover ===")
display(df_turnover.head())

print("=== night ===")
display(df_night.head())


In [ ]:
print(sorted(df_turnover["prefecture"].unique()))
print(sorted(df_night["prefecture"].unique()))

In [ ]:
df_master = df_turnover.merge(df_night, on="prefecture", how="left")

df_master.head()

In [ ]:
print("shape:", df_master.shape)
print("\n欠損確認")
print(df_master.isna().sum())

print("\n重複確認")
print(df_master["prefecture"].duplicated().sum())

In [ ]:
EXCLUDE = ["無回答・不明"]

df_master = df_master[~df_master["prefecture"].isin(EXCLUDE)].copy()

In [ ]:
print(df_master.shape)
print(df_master.isna().sum())


In [ ]:
OUT_DIR = BASE_DIR / "data" / "processed"
OUT_DIR.mkdir(exist_ok=True)

In [ ]:
df_master.to_csv(
    OUT_DIR / "master_nurse_turnover.csv",
    index=False,
    encoding="utf-8-sig"
)